In [7]:
from lightgbm import LGBMRegressor
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.metrics import mean_squared_error
import pickle
from sklearn.svm import SVR

/Users/scai/anaconda3/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


In [2]:
seed = 42

# Load data

In [4]:
meta = pd.read_csv("../data/E0022_P01-P05_sample_map.txt", sep='\t')
# protein_raw = pickle.load(open("../data/protein/E0022_P06_log_transformed_peptide.pkl", "rb"))
protein_raw = pd.read_csv("../data/E0022_P06_final_sample_map.txt", sep='\t')
ic50 = pd.read_csv("../data/drug/DrugResponse_PANCANCER_GDSC1_GDSC2_IC_20191119.csv")
ic50 = ic50.sort_values(by=['Dataset version'])
ic50 = ic50.drop_duplicates(
    ['Drug Id', 'Cell line name'], keep='last').sort_values(
        by=['Drug Id', 'Cell line name']).reset_index(drop=True)


protein_raw = protein_raw.rename(columns={'Unnamed: 0': 'Automatic_MS_filename'})
protein_raw_merge = pd.merge(protein_raw, meta[['Automatic_MS_filename', 'Cell_line']])

protein_sample_avg = protein_raw_merge.drop(['Automatic_MS_filename'],
                                      axis=1).groupby(['Cell_line']).agg(np.nanmean).reset_index()

# two drugs - 201 and 1001

## 201 - most effective

In [6]:
drug_id = 201
tmp_df = pd.merge(
    protein_sample_avg,
    ic50[ic50['Drug Id'] == drug_id][['Cell line name', 'IC50']],
    how='inner',
    left_on='Cell_line',
    right_on='Cell line name')

X = tmp_df.drop(['Cell_line', 'Cell line name', 'IC50'], axis=1)
y = tmp_df['IC50']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)

In [9]:
params = {'C': [1e5, 1e6], 'gamma': [1e-10, 1e-9]}
rcv = GridSearchCV(SVR(), params, scoring='neg_mean_absolute_error', cv=5, n_jobs=-1)
rcv.fit(X, y)

GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=SVR(C=1.0, cache_size=200, coef0=0.0, degree=3,
                           epsilon=0.1, gamma='auto_deprecated', kernel='rbf',
                           max_iter=-1, shrinking=True, tol=0.001,
                           verbose=False),
             iid='warn', n_jobs=-1,
             param_grid={'C': [100000.0, 1000000.0], 'gamma': [1e-10, 1e-09]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring='neg_mean_absolute_error', verbose=0)

In [12]:
[rcv.cv_results_[f'split{i}_test_score'][rcv.best_index_] for i in range(5)]

[-1.5085251231906023,
 -1.399607793604044,
 -1.4528304436687496,
 -1.503865210826909,
 -1.4912633450340462]

In [43]:
mean_squared_error(y_test, y_pred)

3.364149487082684

## 1001

In [45]:
drug_id = 1001
tmp_df = pd.merge(
    protein_sample_avg,
    ic50_shuffle[ic50_shuffle['Drug Id'] == drug_id][['Cell line name', 'IC50']],
    how='inner',
    left_on='Cell_line',
    right_on='Cell line name')

X = tmp_df.drop(['Cell_line', 'Cell line name', 'IC50'], axis=1)
y = tmp_df['IC50']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)

In [46]:
reg = LGBMRegressor()
reg.fit(X_train, y_train)

y_pred = reg.predict(X_test)

In [47]:
mean_squared_error(y_test, y_pred)

1.2540908072460701